In [1]:
import torch
import numpy as np
from torch import linalg as LA
import SimpleITK as sitk

# torch.set_default_dtype(torch.float64)

use_cuda = 1
torchdeviceId = torch.device('cuda:0') if use_cuda else 'cpu'
torchdtype = torch.double

In [2]:
torchdeviceId

device(type='cuda', index=0)

In [3]:
def checkNaN(A):
    if (A != A).any():
        print('NaN')
        return(np.argwhere(np.isnan(A.cpu().numpy())))
    else:
        print('Good')

def check_L2_diff_error(a,b):
    L2Diff = LA.norm(a - b)
    L2Error =  2*LA.norm(a - b)/( LA.norm(a)+LA.norm(b))
    return L2Diff, L2Error

def check_eigvec(a):
    return abs(torch.einsum('...i,...i->...',a[:,:,0],a[:,:,1]))+\
            abs(torch.einsum('...i,...i->...',a[:,:,2],a[:,:,1]))+\
            abs(torch.einsum('...i,...i->...',a[:,:,0],a[:,:,2]))

In [4]:
W = sitk.GetArrayFromImage(sitk.ReadImage(f'W.nhdr'))

In [5]:
W=torch.from_numpy(W).to(torchdeviceId)

In [8]:
from torch_sym3eig import Sym3Eig
val1, vec1 = Sym3Eig.apply(W)
val2, vec2 = Sym3Eig.apply(W)
val3, vec3 = Sym3Eig.apply(W)

In [9]:
torch.where(vec2-vec3!=0)

(tensor([], device='cuda:0', dtype=torch.int64),
 tensor([], device='cuda:0', dtype=torch.int64),
 tensor([], device='cuda:0', dtype=torch.int64))

In [10]:
index = checkNaN(torch.sum(vec2,(1,2)))

Good


In [11]:
torch.where(torch.abs(check_eigvec(vec1))>1e-6)[0]

tensor([ 70169,  70170,  70171,  ..., 388003, 388004, 388044], device='cuda:0')

In [29]:
idx =165664 #215765#273002#187106#190506#186165#178612##91978#215765#166523

In [30]:
W[idx]

tensor([[34.9841, 25.0000,  0.0000],
        [25.0000, 35.0159,  0.0000],
        [ 0.0000,  0.0000, 10.0000]], device='cuda:0', dtype=torch.float64)

In [31]:
vec1[idx]

tensor([[-0.7069,  0.0000, -0.7073],
        [-0.7073,  0.0000,  0.7069],
        [-0.0000,  1.0000,  0.0000]], device='cuda:0', dtype=torch.float64)

In [32]:
vec2[idx]

tensor([[-0.7069,  0.0000, -0.7073],
        [-0.7073,  0.0000,  0.7069],
        [-0.0000,  1.0000,  0.0000]], device='cuda:0', dtype=torch.float64)

In [33]:
vec3[idx]

tensor([[-0.7069,  0.0000, -0.7073],
        [-0.7073,  0.0000,  0.7069],
        [-0.0000,  1.0000,  0.0000]], device='cuda:0', dtype=torch.float64)

In [34]:
_ , vec_torch = torch.linalg.eig(W)

In [35]:
vec_torch[idx]

tensor([[-0.7073+0.j, -0.7069+0.j,  0.0000+0.j],
        [ 0.7069+0.j, -0.7073+0.j,  0.0000+0.j],
        [ 0.0000+0.j,  0.0000+0.j,  1.0000+0.j]], device='cuda:0',
       dtype=torch.complex128)

In [36]:
torch.max(torch.det(vec1)-torch.det(vec_torch.real))

tensor(6.6613e-16, device='cuda:0', dtype=torch.float64)

In [28]:
torch.max(vec1[:,:,2]-vec_torch[:,:,1].real)
torch.where(vec1[:,:,2]-vec_torch[:,:,1].real>1.4142)

(tensor([165656, 165664, 322030, 322032, 322036, 322038, 343717, 343718, 343719,
         343720, 343721, 343722, 343723, 343724, 343725, 343726, 343727, 343728,
         343729, 347817, 347818, 347819, 347820, 347821, 347822, 347823, 347824,
         347825, 347826, 347827, 347828, 347829, 351917, 351918, 351919, 351920,
         351921, 351922, 351923, 351924, 351925, 351926, 351927, 351928, 351929,
         356017, 356018, 356019, 356020, 356021, 356022, 356023, 356024, 356025,
         356026, 356027, 356028, 356029, 360118, 360119, 360120, 360121, 360122,
         360123, 360124, 360125, 360126, 360127, 360128, 364219, 364220, 364221,
         364222, 364223, 364224, 364225, 364226, 364227, 368321, 368322, 368323,
         368324, 368325], device='cuda:0'),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [27]:
torch.max(vec1[:,:,1]-vec_torch[:,:,2].real)

tensor(1., device='cuda:0', dtype=torch.float64)

In [27]:
vec_torch[idx].real-vec3[idx]

tensor([[-5.7339e-09, -3.2771e-03, -3.2771e-03],
        [ 1.7501e-06,  9.9999e-01,  9.9999e-01],
        [ 0.0000e+00,  1.0000e+00,  1.0000e+00]], device='cuda:0',
       dtype=torch.float64)

In [23]:
AlamdaI = W[idx]-val1[idx,1]*torch.eye(3).to(torchdeviceId)
AlamdaI

tensor([[ 0.9091,  6.6804,  0.0000],
        [ 6.6804, 49.0909,  0.0000],
        [ 0.0000,  0.0000,  0.0000]], device='cuda:0', dtype=torch.float64)

In [24]:
v = torch.cross(AlamdaI[:,0],vec1[idx,:,2])
v/torch.norm(v)

tensor([ 0.9909, -0.1348,  0.0000], device='cuda:0', dtype=torch.float64)

In [31]:
torch.dot(v/torch.norm(v),vec1[idx,:,2])

tensor(-1.4204e-08, device='cuda:0', dtype=torch.float64)

--------------------------------------------------

In [37]:
logm_tor = sitk.GetArrayFromImage(sitk.ReadImage(f'logm_tor.nhdr'))
logm_sym = sitk.GetArrayFromImage(sitk.ReadImage(f'logm_sym.nhdr'))

In [44]:
np.where(abs(logm_tor-logm_sym)>5e-06)

(array([359175, 359185, 367375, 367376, 367384, 367385]),
 array([1, 1, 1, 1, 1, 1]),
 array([0, 0, 0, 0, 0, 0]))

In [47]:
index = 367376
logm_tor[index]-logm_sym[index]

array([[ 9.03234412e-08, -1.09350114e-09,  0.00000000e+00],
       [ 5.09198546e-06, -6.16461442e-08,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -2.86772956e-08]])

--------------------------------------------------

In [5]:
file_name = [1,2]
input_dir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/working_3d_python'
tensor_lin_list, tensor_met_list, mask_list, mask_thresh_list = [], [], [], []
height, width, depth = 100,100,41

for s in range(len(file_name)):
#     read tensor and mask files
    tensor_np = sitk.GetArrayFromImage(sitk.ReadImage(f'{input_dir}/cubic{file_name[s]}_orig_tensors.nhdr'))
    mask_np = sitk.GetArrayFromImage(sitk.ReadImage(f'{input_dir}/cubic{file_name[s]}_filt_mask.nhdr'))
    tensor_lin_list.append(torch.from_numpy(tensor_np).double().permute(3,2,1,0))
#     create union of masks
#     mask_union += torch.from_numpy(mask_np).double().permute(2,1,0).to(device)
    mask_list.append(torch.from_numpy(mask_np).double().permute(2,1,0))
#     rearrange tensor_lin to tensor_met
    tensor_met_zeros = torch.zeros(height,width,depth,3,3,dtype=torch.float64)
    tensor_met_zeros[:,:,:,0,0] = tensor_lin_list[s][0]
    tensor_met_zeros[:,:,:,0,1] = tensor_lin_list[s][1]
    tensor_met_zeros[:,:,:,0,2] = tensor_lin_list[s][2]
    tensor_met_zeros[:,:,:,1,0] = tensor_lin_list[s][1]
    tensor_met_zeros[:,:,:,1,1] = tensor_lin_list[s][3]
    tensor_met_zeros[:,:,:,1,2] = tensor_lin_list[s][4]
    tensor_met_zeros[:,:,:,2,0] = tensor_lin_list[s][2]
    tensor_met_zeros[:,:,:,2,1] = tensor_lin_list[s][4]
    tensor_met_zeros[:,:,:,2,2] = tensor_lin_list[s][5]
    tensor_met_list.append(torch.inverse(tensor_met_zeros))

In [6]:
file_name = [105923,103818]
input_dir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/working_3d_python'
output_dir = 'BrainAtlas2'
height, width, depth = 145,174,145
sample_num = len(file_name)
tensor_lin_list, tensor_met_list, mask_list, mask_thresh_list, fa_list = [], [], [], [], []
mask_union = torch.zeros(height, width, depth).double()
phi_inv_acc_list, phi_acc_list, energy_list = [], [], []
resume = False

for s in range(len(file_name)):
# for s in range(1):
#     read tensor and mask files
    tensor_np = sitk.GetArrayFromImage(sitk.ReadImage(f'{input_dir}/{file_name[s]}/scaled_tensors.nhdr'))
    mask_np = sitk.GetArrayFromImage(sitk.ReadImage(f'{input_dir}/{file_name[s]}/filt_mask.nhdr'))
    tensor_lin_list.append(torch.from_numpy(tensor_np).double().permute(3,2,1,0))
#     create union of masks
    mask_union += torch.from_numpy(mask_np).double().permute(2,1,0)
    mask_list.append(torch.from_numpy(mask_np).double().permute(2,1,0))
#     rearrange tensor_lin to tensor_met
    tensor_met_zeros = torch.zeros(height,width,depth,3,3,dtype=torch.float64)
    tensor_met_zeros[:,:,:,0,0] = tensor_lin_list[s][0]
    tensor_met_zeros[:,:,:,0,1] = tensor_lin_list[s][1]
    tensor_met_zeros[:,:,:,0,2] = tensor_lin_list[s][2]
    tensor_met_zeros[:,:,:,1,0] = tensor_lin_list[s][1]
    tensor_met_zeros[:,:,:,1,1] = tensor_lin_list[s][3]
    tensor_met_zeros[:,:,:,1,2] = tensor_lin_list[s][4]
    tensor_met_zeros[:,:,:,2,0] = tensor_lin_list[s][2]
    tensor_met_zeros[:,:,:,2,1] = tensor_lin_list[s][4]
    tensor_met_zeros[:,:,:,2,2] = tensor_lin_list[s][5]
#     balance the background and subject by rescaling
#     tensor_met_zeros = tensor_cleaning(tensor_met_zeros, scale_factor=torch.tensor(1,dtype=torch.float64))
#     fa_list.append(fractional_anisotropy(tensor_met_zeros))
    tensor_met_list.append(torch.inverse(tensor_met_zeros))
    fore_back_adaptor = torch.where(torch.det(tensor_met_list[s])>1e1, 5e-4, 1.)
    mask_thresh_list.append(fore_back_adaptor)
    tensor_met_list[s] = torch.einsum('ijk...,lijk->ijk...', tensor_met_list[s], mask_thresh_list[s].unsqueeze(0))
#     initialize the accumulative diffeomorphism    
#     if resume==False:
#         print('start from identity')
#         phi_inv_acc_list.append(get_idty(height, width, depth))
#         phi_acc_list.append(get_idty(height, width, depth))
#     else:
#         print('start from checkpoint')
#         phi_inv_acc_list.append(torch.from_numpy(sio.loadmat(f'{output_dir}/brain{file_name[s]}_{start_iter-1}_phi_inv.mat')['diffeo']))
#         phi_acc_list.append(torch.from_numpy(sio.loadmat(f'{output_dir}/brain{file_name[s]}_{start_iter-1}_phi.mat')['diffeo']))
#         tensor_met_list[s] = phi_pullback(phi_inv_acc_list[s], tensor_met_list[s])
#     energy_list.append([])    
    
mask_union[mask_union>0] = 1

In [7]:
# print out the nonPDS matrix and the determinant
if torch.min(torch.det(g0))<0:
    g = g0
    detg = torch.det(g)
    I1,I2,I3 = (detg0==torch.min(detg)).nonzero().reshape(-1)
    print('g0 is not PD')
    print(g0[I1,I2,I3])
    print('The det is', g0[I1,I2,I3].det().item())
elif torch.min(torch.det(g1))<0:
    g = g1
    detg = torch.det(g)
    I1,I2,I3 = (detg0==torch.min(detg)).nonzero().reshape(-1)
    print('g1 is not PD')
    print(g1[I1,I2,I3])
    print('The det is',g1[I1,I2,I3].det())

In [8]:
# A, B = g0, g1
A, B = tensor_met_list[0], tensor_met_list[1]

In [9]:
%%time
G = LA.cholesky(A)
G_inv = torch.inverse(G)
W = torch.einsum("...ik,...ks,...js->...ij",[G_inv, B, G_inv])

CPU times: user 11.1 s, sys: 2.18 s, total: 13.3 s
Wall time: 1.26 s


In [10]:
torch.sum(LA.cholesky(W))

tensor(11529832.9221, dtype=torch.float64)

In [60]:
import math

In [53]:
def eigenvalues(M):
    a11 = M[:,0,0]
    a12 = M[:,0,1]
    a13 = M[:,0,2]
    a22 = M[:,1,1]
    a23 = M[:,1,2]
    a33 = M[:,2,2]
    c2 = -a11-a22-a33
    c1 = a11*a22+a11*a33+a22*a33-torch.pow(a12,2)-torch.pow(a13,2)-torch.pow(a23,2)
    c0 = a11*torch.pow(a23,2)+a22*torch.pow(a13,2)+a33*torch.pow(a12,2)-a11*a22*a33-2*a13*a12*a23
    p = torch.pow(c2,2)-3*c1
    q = -13.5*c0-torch.pow(c2,3)+4.5*c2*c1
    t = 2*torch.pow(p,-1.5)*q
    phi = torch.arctan(torch.sqrt(27*(torch.pow(c1,2)*(p-c1)/4.+c0*(q+27./4.*c0)))/q)/3.
    x1 = 2*torch.cos(phi)
    x2 = -torch.cos(phi)-math.sqrt(3)*torch.sin(phi)
    x3 = -torch.cos(phi)+math.sqrt(3)*torch.sin(phi)
    lamda1 = torch.sqrt(p)*x1/3.-c2/3.
    lamda2 = torch.sqrt(p)*x2/3.-c2/3.
    lamda3 = torch.sqrt(p)*x3/3.-c2/3.
    return lamda1, lamda2, lamda3

In [54]:
eigenvalues(g.unsqueeze(0))

(tensor([0.1278], dtype=torch.float64),
 tensor([0.0445], dtype=torch.float64),
 tensor([0.0444], dtype=torch.float64))